In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
dtest = pd.read_csv("train.csv")

ftest = pd.read_csv("test.csv")

dtest.drop("Name", axis=1, inplace = True)

In [5]:
y = dtest.Transported
features = ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

X = dtest[features]
X2 = ftest[features]

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]


In [17]:
OHcols = ["CryoSleep", "VIP", "Destination", "HomePlanet"]

cols2 = ["Cabin", "PassengerId"]
dtest2 = dtest[cols2]
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
#Up here is changing the amount of rows
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[OHcols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[OHcols]))
OH_cols_final = pd.DataFrame(OH_encoder.fit_transform(X2[OHcols]))


# Putting index bacl
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index
OH_cols_final.index = X2.index

# Removing previous columns
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)
num_final = X2.drop(object_cols, axis = 1)

# Adding to other columns
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
OH_X_final = pd.concat([num_final, OH_cols_final], axis=1)
# Imputing the numerical columns
my_imputer = SimpleImputer()
Mid_X_train = pd.DataFrame(my_imputer.fit_transform(OH_X_train))
Mid_X_valid = pd.DataFrame(my_imputer.transform(OH_X_valid))
Mid_final = pd.DataFrame(my_imputer.transform(OH_X_final))

# Imputation removed column names; put them back
Mid_X_train.columns = OH_X_train.columns
Mid_X_valid.columns = OH_X_valid.columns
Mid_final.columns = OH_X_final.columns

Mid_X_train.isna().sum()


/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', '

Age             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
0               0
1               0
2               0
3               0
4               0
5               0
6               0
7               0
8               0
9               0
10              0
11              0
12              0
13              0
dtype: int64

In [18]:
model = GradientBoostingClassifier()
model.fit(Mid_X_train, y_train)

/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


GradientBoostingClassifier()

In [26]:
# for n in [100, 125, 150, 200]:
#     model = GradientBoostingClassifier(random_state = 1, max_depth = 4, n_estimators  = n)
#     model.fit(Mid_X_train, y_train)
#     preds_valid = model.predict(Mid_X_valid)
#     print(n, accuracy_score(y_valid, preds_valid))

In [24]:
train_final = pd.concat([Mid_X_valid, Mid_X_train])
y_final = pd.concat([y_valid, y_train])
model.predict(train_final)
y_final.isna().sum()

model_final = GradientBoostingClassifier(random_state = 1, max_depth = 4)
model_final.fit(train_final, y_final)
preds = model_final.predict(Mid_final)


/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [25]:
output = pd.DataFrame({"PassengerId": ftest.PassengerId, "Transported": preds})
output.to_csv('submission5.csv', index = False)

1527
Sarah Braverman
0.79354
Your Best Entry!
Your most recent submission scored 0.79354, which is an improvement of your previous score of 0.79167. Great job!